# Hunter Lebow
# ANOP370

<h2>[Textbook (adapted), Chapter 3, Problem 22] </h2>

Valu-Com Electronics manufactures five different models of telecommunications interface cards for PCs and laptops. As summarized in the following table, each of these devices requires differing amounts of printed circuit board, resistors, memory chips, and assembly.

In its next production period, Valu-Com has 80,000 square inches of PC board, 100,000 resistors, 30,000 memory chips, and 5,000 hours of assembly time available. The manufacturing cost above includes the cost of the components as well as of the assembly hours. The company wants to maximize their profit over all products sold, which is given for each product as the difference between selling price and manufacturing cost multiplied by the number of units produced.
You should expect questions such as the following:
a) If we are asked to make more of a given product, how does that affect the profit?
b) If manufacturing costs or selling prices change, would we still make the same decisions?
c) If we purchase more supplies or hire more staff for assembly, would that increase profit?
In all of these cases, you should be mindful of the extent to which your answers would remain correct. In other words, very drastic changes might be beyond the scope of the sensitivity report.

In [6]:
import pandas as pd
import gurobipy as gb
df = pd.read_excel("data.xlsx", sheet_name="raw-data").set_index('rows', drop=True)
df


,HyperLink,FastLink,SpeedLink,MicroLink,EtherLink
rows,,,,,
Printed Circuit Board (sq in),20.00,15.0,10.0,8.00,5
Resistors,28.00,24.0,18.0,12.00,16
Memory chips,8.00,8.0,4.0,4.00,6
Assembly hours,0.75,0.6,0.5,0.65,1
Selling price (per unit),189.00,149.0,129.0,169.00,139
Manufacturing Costs (per unit),136.00,101.0,96.0,134.00,101


In [7]:
model = gb.Model()
decision_vars = df.columns
product = model.addVars(len(decision_vars))

sell_price = df.loc["Selling price (per unit)", :]
cogs = df.loc["Manufacturing Costs (per unit)", :]
model.setObjective(
    gb.quicksum(
        product[i]*(sell_price[i]- cogs[i]) for i in range(len(decision_vars))
    ), gb.GRB.MAXIMIZE
)

MAX_PCB = 80000
pcb = df.loc["Printed Circuit Board (sq in)", :]
model.addConstr(
    gb.quicksum(
        product[i]*pcb[i] for i in range(len(decision_vars))
    ) <= MAX_PCB
)

MAX_RESISTORS = 100000
resistors = df.loc["Resistors", :]
model.addConstr(
    gb.quicksum(
        product[i]*resistors[i] for i in range(len(decision_vars))
    ) <= MAX_RESISTORS
)

MAX_MEMORY_CHIPS = 30000
memory_chips = df.loc["Memory chips", :]
model.addConstr(
    gb.quicksum(
        product[i]*memory_chips[i] for i in range(len(decision_vars))
    ) <= MAX_MEMORY_CHIPS
)

MAX_HOURS = 5000
hours = df.loc["Assembly hours", :]
model.addConstr(
    gb.quicksum(
        product[i]*hours[i] for i in range(len(decision_vars))
    ) <= MAX_HOURS
)

model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 5 columns and 20 nonzeros
Model fingerprint: 0xb740ef76
Coefficient statistics:
  Matrix range     [5e-01, 3e+01]
  Objective range  [3e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 1e+05]
Presolve time: 0.00s
Presolved: 4 rows, 5 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.7500000e+31   1.363125e+31   8.750000e+01      0s
       2    2.6250000e+05   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.625000000e+05


In [8]:
print(f"\nOptimal units produced for each product to maximize company profit: \n")
if model.Status == gb.GRB.Status.OPTIMAL:
    for v in model.getVars():
        print(f"{df.columns[v.index]}: {v.x} units produced")
print(f"\nTotal Profit: ${round(model.ObjVal, 2)}")


Optimal units produced for each product to maximize company profit: 

HyperLink: 0.0 units produced
FastLink: 0.0 units produced
SpeedLink: 0.0 units produced
MicroLink: 7500.0 units produced
EtherLink: 0.0 units produced

Total Profit: $262500.0


In [9]:
excel_model = pd.read_excel("data.xlsx", sheet_name="Model")
excel_model

,rows,Printed Circuit Board (sq in),Resistors,Memory chips,Assembly hours,Selling price (per unit),Manufacturing Costs (per unit),Profit (per unit),Amount produced of each product
0,HyperLink,20,28,8,0.75,189,136,53,0
1,FastLink,15,24,8,0.60,149,101,48,0
2,SpeedLink,10,18,4,0.50,129,96,33,0
3,MicroLink,8,12,4,0.65,169,134,35,7500
4,EtherLink,5,16,6,1.00,139,101,38,0
5,Constraints,80000,100000,30000,5000.00,MAX,MIN,MAX,DECISION VARS
6,Totals,60000,90000,30000,4875.00,1267500,1005000,262500,7500


In [10]:
sensitivity_report = pd.read_excel("data.xlsx", sheet_name="Sensitivity Report 1")
display(sensitivity_report)

,Microsoft Excel 16.66 Sensitivity Report,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Worksheet: [data.xlsx]Model,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Report Created: 11/16/22 2:47:00 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Variable Cells,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,Final,Reduced,Objective,Allowable,Allowable
6,NaN,Cell,Name,Value,Cost,Coefficient,Increase,Decrease
7,NaN,$I$2,HyperLink Amount produced of each product,0,-11.666667,53,11.666667,1000000000000000019884624838656
8,NaN,$I$3,FastLink Amount produced of each product,0,-16,48,16,1000000000000000019884624838656
9,NaN,$I$4,SpeedLink Amount produced of each product,1666.666667,0,33,15,1.0


a) If we are asked to make more of a given product, how does that affect the profit?
- Look at "Reduced Cost" to see how producing more of a given product reduces cost from the optimum.

b) If manufacturing costs or selling prices change, would we still make the same decisions?
- No, these are decisions variables which affect the equation for the optimal solution.

c) If we purchase more supplies or hire more staff for assembly, would that increase profit?
- It depends which kinds of supplies and how many.  Hiring more staff for assembly will decrease assembly time per unit which will raise profits

In all of these cases, you should be mindful of the extent to which your answers would remain correct. In other words, very drastic changes might be beyond the scope of the sensitivity report.



<a href=https://www.excel-easy.com/examples/sensitivity-analysis.html >
More about Reduced Cost and Shadow Pricing on Microsoft Excel Sensitivy Reports
</a>